# Chapter 13

Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [26]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [27]:
import thinkstats2
import thinkplot
import numpy as np
import pandas as pd

In [28]:
try:
    import empiricaldist
except ImportError:
    !pip install empiricaldist

In [29]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)
import nsfg

preg = nsfg.ReadFemPreg()
complete = preg.query("outcome in [1, 3, 4]").prglngth
cdf = thinkstats2.Cdf(complete, label="cdf")

In [31]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/survival.py")

import survival


def MakeSurvivalFromCdf(cdf, label=""):
    """Makes a survival function based on a CDF.

    cdf: Cdf

    returns: SurvivalFunction
    """
    ts = cdf.xs
    ss = 1 - cdf.ps
    return survival.SurvivalFunction(ts, ss, label)

sf = MakeSurvivalFromCdf(cdf, label="survival")
hf = sf.MakeHazardFunction(label="hazard")

/Users/madisonchristiansen/Desktop/DSC/survival.py:71: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=self.ts)


In [39]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz"
)

resp6 = nsfg.ReadFemResp()

from collections import Counter


def EstimateHazardFunction(complete, ongoing, label="", verbose=False):
    """Estimates the hazard function by Kaplan-Meier.

    http://en.wikipedia.org/wiki/Kaplan%E2%80%93Meier_estimator

    complete: list of complete lifetimes
    ongoing: list of ongoing lifetimes
    label: string
    verbose: whether to display intermediate results
    """
    if np.sum(np.isnan(complete)):
        raise ValueError("complete contains NaNs")
    if np.sum(np.isnan(ongoing)):
        raise ValueError("ongoing contains NaNs")

    hist_complete = Counter(complete)
    hist_ongoing = Counter(ongoing)

    ts = list(hist_complete | hist_ongoing)
    ts.sort()

    at_risk = len(complete) + len(ongoing)

    lams = pd.Series(index=ts, dtype=float)
    for t in ts:
        ended = hist_complete[t]
        censored = hist_ongoing[t]

        lams[t] = ended / at_risk
        if verbose:
            print(t, at_risk, ended, censored, lams[t])
        at_risk -= ended + censored

    return survival.HazardFunction(lams, label=label)
EstimateHazardFunction(complete, ongoing, label="", verbose=False)

Downloaded 2002FemResp.dct
Downloaded 2002FemResp.dat.gz


NameError: name 'ongoing' is not defined

In [40]:
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/1995FemRespData.dat.gz"
)
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2006_2010_FemRespSetup.dct"
)
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2006_2010_FemResp.dat.gz"
)

resp5 = survival.ReadFemResp1995()
resp6 = survival.ReadFemResp2002()
resp7 = survival.ReadFemResp2010()
resps = [resp5, resp6, resp7]

Downloaded 1995FemRespData.dat.gz
Downloaded 2006_2010_FemRespSetup.dct
Downloaded 2006_2010_FemResp.dat.gz


## Exercises

**Exercise:**    In NSFG Cycles 6 and 7, the variable `cmdivorcx` contains the date of divorce for the respondent’s first marriage, if applicable, encoded in century-months.

Compute the duration of marriages that have ended in divorce, and the duration, so far, of marriages that are ongoing. Estimate the hazard and survival curve for the duration of marriage.

Use resampling to take into account sampling weights, and plot data from several resamples to visualize sampling error.

Consider dividing the respondents into groups by decade of birth, and possibly by age at first marriage.

In [32]:
from __future__ import print_function

import pandas
import numpy as np
import nsfg
import thinkplot
import thinkstats2

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/survival.py")
import survival

In [108]:
def clean(resp): 
    resp.cmdivorcx.replace()
    
    resp['NotDivorced'] = resp.cmdivorcx.isnull().astype(int)
    resp['Duration'] = (resp.cmdivorcx - resp.cmmarrhx) 
    resp['DurationSoFar'] = (resp.cmintvw - resp.cmmarrhx) 

    month0 = pandas.to_datetime('1899-12-15')
    dates = [month0 + pandas.DateOffset(months=cm) 
             for cm in resp.cmbirth]
    resp['Decade'] = (pandas.DatetimeIndex(dates).year - 1900)

In [109]:
def ResampDivorceCurve(resps):
    for _ in range(41):
        samples = [thinkstats2.ResampleRowsWeighted(resp) 
                   for resp in resps]
        sample = pandas.concat(samples, ignore_index=True)

In [110]:
def ResampDivorceCurvebyDecade(resps):
    for i in range(41):
        samples = [thinkstats2.ResampleRowsWeighted(resp) 
                   for resp in resps]
        sample = pandas.concat(samples, ignore_index=True)
        groups = sample.groupby('Decade')
        EstSurvivalbyDecade(groups, alpha=0.1)

In [111]:
def EstSurvivalbyDecade(groups, **options):
    for name, group in groups:
        print(name, len(group))
        _, sf = EstSurvival(group)

In [112]:
def EstSurvival(resp):
    complete = resp[resp.NotDivorced == 0].Duration.dropna()
    ongoing = resp[resp.NotDivorced == 1].DurationSoFar.dropna()
    hf = survival.EstimateHazardFunction(complete, ongoing)
    sf = hf.MakeSurvival()

    return hf, sf

In [113]:
resp6 = survival.ReadFemResp2002()
clean(resp6)
married6 = resp6[resp6.evrmarry == 1]

resp7 = survival.ReadFemResp2010()
clean(resp7)
married7 = resp7[resp7.evrmarry == 1]

ResampDivorceCurvebyDecade([married6, married7])

57 68
58 251
59 185
60 248
61 236
62 359
63 362
64 476
65 510
66 503
67 540
68 480
69 522
70 477
71 456
72 413
73 400
74 373
75 340
76 325
77 331
78 343
79 277
80 286
81 269
82 155
83 146
84 107
85 67
86 75
87 39
88 14
89 15
90 10
91 1
92 1
57 74
58 228
59 224
60 219
61 235
62 326
63 390
64 492
65 499
66 424
67 514
68 466
69 512
70 510
71 499
72 404
73 386
74 345
75 364
76 344
77 337
78 378
79 307
80 281
81 247
82 176
83 154
84 120
85 49
86 82
87 48
88 13
89 10
90 2
92 1
57 73
58 217
59 199
60 256
61 267
62 322
63 390
64 482
65 511
66 509
67 529
68 485
69 464
70 522
71 461
72 387
73 435
74 341
75 366
76 339
77 305
78 342
79 290
80 262
81 251
82 179
83 142
84 110
85 64
86 83
87 39
88 16
89 11
90 8
91 2
92 1
57 65
58 242
59 212
60 220
61 257
62 380
63 397
64 501
65 463
66 435
67 480
68 535
69 517
70 504
71 443
72 379
73 429
74 353
75 376
76 343
77 320
78 344
79 286
80 287
81 248
82 179
83 139
84 111
85 64
86 77
87 43
88 17
89 8
90 6
57 72
58 228
59 233
60 223
61 263
62 287
63 387
64 482


/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonch

57 69
58 219
59 198
60 243
61 255
62 332
63 405
64 466
65 479
66 495
67 518
68 521
69 518
70 512
71 468
72 416
73 376
74 349
75 362
76 329
77 316
78 338
79 281
80 293
81 252
82 193
83 132
84 106
85 54
86 81
87 47
88 15
89 15
90 5
91 1
92 1
57 71
58 219
59 226
60 243
61 268
62 325
63 370
64 486
65 461
66 491
67 485
68 507
69 512
70 498
71 498
72 394
73 368
74 378
75 370
76 311
77 303
78 347
79 310
80 283
81 270
82 172
83 162
84 101
85 66
86 77
87 43
88 14
89 19
90 8
91 4
57 58
58 268
59 216
60 232
61 263
62 355
63 386
64 461
65 489
66 471
67 505
68 490
69 490
70 525
71 459
72 433
73 394
74 352
75 383
76 359
77 301
78 329
79 268
80 257
81 264
82 172
83 137
84 112
85 59
86 91
87 43
88 19
89 9
90 7
91 3
57 63
58 241
59 234
60 265
61 221
62 344
63 444
64 492
65 495
66 452
67 478
68 520
69 487
70 490
71 464
72 428
73 385
74 344
75 317
76 364
77 303
78 358
79 300
80 261
81 292
82 169
83 142
84 123
85 57
86 61
87 35
88 13
89 10
90 4
91 2
92 2
57 74
58 238
59 216
60 258
61 223
62 298
63 401
64 

/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonch

57 64
58 221
59 216
60 241
61 242
62 311
63 386
64 476
65 491
66 448
67 508
68 534
69 506
70 540
71 471
72 426
73 387
74 354
75 355
76 369
77 308
78 315
79 302
80 265
81 253
82 200
83 152
84 107
85 66
86 66
87 44
88 16
89 13
90 5
91 1
92 1
57 85
58 240
59 226
60 231
61 254
62 342
63 403
64 511
65 480
66 482
67 489
68 514
69 516
70 486
71 472
72 389
73 418
74 374
75 328
76 313
77 298
78 323
79 318
80 262
81 258
82 172
83 144
84 119
85 68
86 75
87 44
88 6
89 13
90 5
91 1
92 1
57 63
58 264
59 197
60 231
61 237
62 320
63 377
64 479
65 465
66 490
67 525
68 522
69 553
70 508
71 470
72 372
73 382
74 358
75 350
76 356
77 316
78 335
79 284
80 280
81 241
82 211
83 145
84 110
85 56
86 82
87 51
88 12
89 10
90 4
91 3
92 1
57 57
58 242
59 228
60 244
61 229
62 324
63 420
64 513
65 481
66 516
67 492
68 500
69 531
70 481
71 495
72 426
73 402
74 365
75 354
76 352
77 294
78 340
79 270
80 253
81 264
82 161
83 125
84 113
85 47
86 59
87 50
88 7
89 15
90 7
91 2
92 1
57 65
58 232
59 182
60 223
61 239
62 317
6

...

/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonch

57 65
58 225
59 209
60 259
61 255
62 350
63 394
64 512
65 501
66 464
67 476
68 533
69 505
70 489
71 468
72 387
73 371
74 363
75 357
76 353
77 331
78 325
79 296
80 295
81 259
82 138
83 135
84 121
85 54
86 81
87 53
88 11
89 18
90 5
91 1
92 1
57 63
58 242
59 206
60 227
61 250
62 355
63 389
64 490
65 462
66 496
67 522
68 538
69 507
70 476
71 443
72 426
73 390
74 362
75 372
76 331
77 315
78 332
79 296
80 274
81 268
82 173
83 137
84 122
85 69
86 59
87 37
88 9
89 16
90 6
57 59
58 239
59 238
60 246
61 268
62 352
63 402
64 448
65 464
66 464
67 501
68 462
69 517
70 508
71 460
72 417
73 417
74 380
75 341
76 342
77 325
78 351
79 284
80 268
81 253
82 167
83 153
84 109
85 64
86 84
87 46
88 16
89 8
90 5
91 2
57 67
58 262
59 213
60 231
61 248
62 329
63 369
64 479
65 502
66 508
67 499
68 493
69 517
70 542
71 479
72 421
73 414
74 343
75 323
76 300
77 291
78 342
79 321
80 269
81 268
82 192
83 120
84 102
85 71
86 85
87 37
88 7
89 9
90 7
57 64
58 240
59 215
60 242
61 255
62 358
63 411
64 465
65 506
66 482


/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonch

69 511
70 504
71 479
72 389
73 348
74 351
75 344
76 339
77 309
78 351
79 271
80 290
81 283
82 171
83 142
84 112
85 66
86 74
87 44
88 17
89 10
90 5
57 67
58 258
59 203
60 247
61 240
62 343
63 381
64 468
65 482
66 469
67 509
68 484
69 502
70 514
71 475
72 402
73 400
74 369
75 370
76 324
77 304
78 364
79 290
80 283
81 229
82 203
83 161
84 119
85 64
86 62
87 43
88 15
89 12
90 3
91 1


/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonchristiansen/Desktop/DSC/survival.py:300: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  lams = pd.Series(index=ts)
/Users/madisonch